# [Digital image steganography: Survey and analysis of current methods](http://www.sciencedirect.com/science/article/pii/S0165168409003648)

## DCT
* Li and Wang modified the quantization table itself, and then inserted hidden bits in the middle frequency coefficients. This gave them 36 coefficients to embed in per block, and thus a good payload. [39]
![Modified QT](QT.png)
* JSteg is easily detected with the chi squared test. [3]
* OutGuess used a pRNG to select DCT coefficients, which thwarts the chi squared test. However, an extended version of the chi squared test may work. [3]
* F5 embeds only into non-zero AC DCT coeefficients by decreasing the absolute value of the coefficient by 1. The same bit has to be re-embedded in case the original is 1 or -1. Fridrich et al. came up with a detection method for it. [33]
* Perturbed Quantization has minimal distortion but low capacity. [51]

## Adaptive
Analyzes statistical global features of the cover image, and tries to embed in noisy areas.
* MB1 is model based [58]. Can be broken using first-order statistics [63]. Can also be detected through difference in "blockiness" [64]. Enhanced version to avoid blockiness is MB2 [58]. MB2 can still be detected.
* Chin-Chen et al. estimate the degree of smoothness between pixels, with high capacity. [65]
* A Block Complexity based Data Embedding replaces suitable noisy blocks of pixels with noisy blocks containing data. [66]
* Genetic algorithm breaks detection by artificially counterfeiting statistical features. Time complexity may be an issue. [74]
* Genetic algorithm that uses +/-1 technique on JPEG. [64]
* Kong et al. calculates entropy for each region, which dictates embedding strength of LSB. [76]

## Other 2D image
* Discrete wavelet transform [55] [57] [67]
* Nakamura and Zhao morphing process [79]
* Zeki and Azizah intermediate significant bit algorithm [80]

* New detection methods utilize neural networks [46] [56]

## 3D graphics
* Chao et al. [77]
* Bogomjakov et al. [78]

## Video
* McKeon used 2D DFT for steganography in movies. [44]

## List of tools
![List of tools](Tools.PNG)